In [33]:
import warnings
from os.path import expanduser
import numpy as np
import netCDF4
import rasters as rt

In [11]:
filename = "~/data/VNP21IMG_NRT.A2025147.1700.002.2025147190500.nc"

In [12]:
with netCDF4.Dataset(expanduser(filename)) as file:
    print(file)

<class 'netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    AlgorithmType: Single-Channel
    AlgorithmVersion: VIIRS 2022
    Conventions: CF-1.6
    DayNightFlag: Day
    EastBoundingCoordinate: -34.28002
    EndTime: 2025-05-27 17:06:00.000
    GRingPointLatitude: [ -6.4201703  -2.273785  -22.595377  -27.194231 ]
    GRingPointLongitude: [-67.86452 -40.55275 -34.28002 -64.56987]
    GRingPointSequenceNo: [1 2 3 4]
    InstrumentShortname: VIIRS
    LUTs used: GEOS.it.asm.asm_inst_3hr_glo_L576x361_p42.GEOS5294.2025-05-27T0600.V01.nc4, GEOS.it.asm.asm_inst_3hr_glo_L576x361_p42.GEOS5294.2025-05-27T0900.V01.nc4
    LocalGranuleID: VNP21IMG_NRT.A2025147.1700.002.2025147190500.nc
    LongName: VIIRS/NPP Land Surface Temperature and Emissivity 6-Min L2 Swath 375m
    NWPSource: GEOS5
    NorthBoundingCoordinate: -2.273785
    OrbitNumber: 70374
    PGEVersion: 2.0.5
    PGE_EndTime: 2025-05-27 17:06:00.000
    PGE_Name: PGE609
    PGE_StartTime: 2025-05-27 17:00:00.000

In [15]:
with netCDF4.Dataset(expanduser(filename)) as file:
    print(file.groups)

{'VIIRS_I5_LST': <class 'netCDF4.Group'>
group /VIIRS_I5_LST:
    dimensions(sizes): 
    variables(dimensions): 
    groups: Data Fields, Geolocation Fields}


In [16]:
with netCDF4.Dataset(expanduser(filename)) as file:
    print(file["VIIRS_I5_LST"].groups)

{'Data Fields': <class 'netCDF4.Group'>
group /VIIRS_I5_LST/Data Fields:
    dimensions(sizes): 
    variables(dimensions): uint8 Emis_I5(number_of_lines, number_of_pixels), uint16 Emis_I5_err(number_of_lines, number_of_pixels), uint16 LST(number_of_lines, number_of_pixels), uint8 LST_err(number_of_lines, number_of_pixels), uint8 QC(number_of_lines, number_of_pixels), uint8 View_angle(number_of_lines, number_of_pixels)
    groups: , 'Geolocation Fields': <class 'netCDF4.Group'>
group /VIIRS_I5_LST/Geolocation Fields:
    dimensions(sizes): 
    variables(dimensions): float32 Latitude(number_of_lines, number_of_pixels), float32 Longitude(number_of_lines, number_of_pixels)
    groups: }


In [17]:
with netCDF4.Dataset(expanduser(filename)) as file:
    print(file["VIIRS_I5_LST/Geolocation Fields"].variables)

{'Latitude': <class 'netCDF4.Variable'>
float32 Latitude(number_of_lines, number_of_pixels)
    _FillValue: -999.0
    add_offset: 0.0
    coordinates: latitude longitude
    format: scaled
    long_name: Latitude data
    scale_factor: 1.0
    units: degrees north
    valid_range: [-90.  90.]
path = /VIIRS_I5_LST/Geolocation Fields
unlimited dimensions: 
current shape = (6464, 6400)
filling off, 'Longitude': <class 'netCDF4.Variable'>
float32 Longitude(number_of_lines, number_of_pixels)
    valid_range: [-180.  180.]
    _FillValue: -999.0
    add_offset: 0.0
    coordinates: latitude longitude
    format: scaled
    long_name: Longitude data
    scale_factor: 1.0
    units: degrees east
path = /VIIRS_I5_LST/Geolocation Fields
unlimited dimensions: 
current shape = (6464, 6400)
filling off}


In [27]:
def read_latitude(filename: str) -> np.ndarray:
    with netCDF4.Dataset(expanduser(filename)) as file:
        dataset = file["VIIRS_I5_LST/Geolocation Fields/Latitude"]
        fill_value = dataset._FillValue

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            array = np.array(dataset)
            
        array = np.where(array == fill_value, np.nan, array)
        
        return array

In [29]:
def read_longitude(filename: str) -> np.ndarray:
    with netCDF4.Dataset(expanduser(filename)) as file:
        dataset = file["VIIRS_I5_LST/Geolocation Fields/Longitude"]
        fill_value = dataset._FillValue

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            array = np.array(dataset)
            
        array = np.where(array == fill_value, np.nan, array)
        
        return array

In [31]:
latitude = read_latitude(filename)
latitude

array([[-22.59537697, -22.5974102 , -22.59944344, ..., -27.19339752,
        -27.19381523, -27.19423103],
       [-22.58819389, -22.59023094, -22.59226608, ..., -27.18597794,
        -27.18639374, -27.18680954],
       [-22.58101273, -22.58305168, -22.58508873, ..., -27.17855835,
        -27.17897415, -27.17938423],
       ...,
       [ -2.28840446,  -2.28970623,  -2.29100633, ...,  -6.43294668,
         -6.43388128,  -6.43482828],
       [ -2.28109503,  -2.28239989,  -2.28370309, ...,  -6.42563438,
         -6.42656374,  -6.42749882],
       [ -2.27378511,  -2.27509308,  -2.27639961, ...,  -6.41831827,
         -6.41923761,  -6.42017031]], shape=(6464, 6400))

In [32]:
longitude = read_longitude(filename)
longitude

array([[-34.28002167, -34.28797531, -34.29592514, ..., -64.55300903,
        -64.56147003, -64.56987   ],
       [-34.28216934, -34.29012299, -34.298069  , ..., -64.5533371 ,
        -64.56176758, -64.57021332],
       [-34.2843132 , -34.29226685, -34.30020905, ..., -64.55362701,
        -64.56210327, -64.57054901],
       ...,
       [-40.55051422, -40.55791855, -40.56531525, ..., -67.84766388,
        -67.85520935, -67.86284637],
       [-40.55163193, -40.55904007, -40.56643677, ..., -67.84858704,
        -67.85610199, -67.86367798],
       [-40.55274963, -40.56015778, -40.56755447, ..., -67.84946442,
        -67.85693359, -67.86451721]], shape=(6464, 6400))

In [34]:
geolocation = rt.RasterGeolocation(x=longitude, y=latitude)
geolocation

{
  "dimensions": {
    "rows": 6464,
    "cols": 6400
  },
  "bbox": {
    "xmin": -67.86451721191406,
    "ymin": -27.194231033325195,
    "xmax": -34.28002166748047,
    "ymax": -2.27378511428833
  },
  "crs": "EPSG:4326",
  "resolution": {
    "cell_width": 0.00423565576960579,
    "cell_height": -0.00423565576960579
  }
}